In [1]:
import numpy as np
import theano
import theano.tensor as T

In [3]:
data = np.load('gibbs-sample.dat.npy')
print ('Shape of data: ', data.shape)

Shape of data:  (50000, 16)


In [2]:
class mpf(object):
    """
    Minimum probability flow
    """
    def __init__(self, input, n, epsilon):
        """
        Initialize the parameters of the MPF network
        """
        
        self.W = theano.shared(value = np.zeros((n, n), dtype = theano.config.floatX), name = 'W', borrow = True)
        self.b = theano.shared(value = np.zeros((1,n)), dtype = theano.config.floatX), name = 'b', borrow = True)
        
        self.z = T.dot(input, self.W) 
        
        self.params = [self.W, self.b]
        self.input = input
        
    def Kcost(data, W, b, epsilon):
        """
        Returns the cost.
        Inputs:
        - data: numpy array of data that contains the different states of the network of size n
        - W: (n, n) numpy array of the weight matrix
        - b: (1, ) numpy array of biases
        - espilon: parameter for the cost
        """
        T.exp(T.dot(x, self.W) + self.b)
        
        
        D = data.shape[0]
        return np.sum(np.exp((0.5 - data) * (data.dot(W) + b))) * (epsilon/D)
    

In [7]:
A = theano.shared(value = np.zeros((2, 2), dtype = theano.config.floatX), name = 'A', borrow = True)

In [17]:
A.get_value()

array([[ 0.,  0.],
       [ 0.,  0.]])

In [19]:
B = T.exp(A)

In [21]:
B.get_parents()

[Elemwise{exp,no_inplace}(A)]

In [25]:
e = theano.function(inputs = [A], outputs= B)

TypeError: Cannot use a shared variable (A) as explicit input. Consider substituting a non-shared variable via the `givens` parameter